In [1]:
sc

In [2]:
#import data

high_school = sc.textFile('DOE_High_School_Directory_2014-2015.csv')
sat_results = sc.textFile('SAT_Results.csv')

# print high_school.count()
# print sat_results.count()

In [3]:
# remove header

header_school = high_school.first()
high_school = high_school.filter(lambda x: x != header_school)

header_sat = sat_results.first()
sat_results = sat_results.filter(lambda x: x != header_sat)

print header_school
print "-----"
print header_sat

dbn,school_name,boro,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,bus,subway,primary_address_line_1,city,state_code,zip,website,total_students,campus_name,school_type,overview_paragraph,program_highlights,language_classes,advancedplacement_courses,online_ap_courses,online_language_courses,extracurricular_activities,psal_sports_boys,psal_sports_girls,psal_sports_coed,school_sports,partner_cbo,partner_hospital,partner_highered,partner_cultural,partner_nonprofit,partner_corporate,partner_financial,partner_other,addtl_info1,addtl_info2,start_time,end_time,se_services,ell_programs,school_accessibility_description,number_programs,priority01,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,Location 1
-----
DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score


In [4]:
# filter and clean data

high_school = high_school.filter(lambda x: len(x)>100)
high_school = high_school.filter(lambda x: x.split(',')[2] in ['Manhattan','Bronx','Brooklyn','Queens','Staten Island'])

high_school = high_school.map(lambda x: (x.split(',')[0], x.split(',')[2]))

high_school_df = sqlContext.createDataFrame(high_school, ('DBN', 'Borough'))

# high_school_df.show()

In [5]:
sat_results = sat_results.filter(lambda x: x.split(',')[-4] != 's')

sat_results = sat_results.map(lambda x: (x.split(',')[0], int(x.split(',')[-4]), int(x.split(',')[-4])*int(x.split(',')[-1])))

sat_results_df = sqlContext.createDataFrame(sat_results, ('DBN', 'Num Test Takers', 'Sum Math'))

# sat_results_df.show()

In [6]:
# join data

join_data = sat_results_df.join(high_school_df, ['DBN'])

join_data.show()

+------+---------------+--------+---------+
|   DBN|Num Test Takers|Sum Math|  Borough|
+------+---------------+--------+---------+
|02M449|             72|   27936|Manhattan|
|15K519|             77|   29645| Brooklyn|
|02M288|             62|   24304|Manhattan|
|10X440|            443|  181630|    Bronx|
|13K595|             89|   38359| Brooklyn|
|14K558|             72|   26208| Brooklyn|
|27Q260|             87|   32451|   Queens|
|07X527|             62|   22506|    Bronx|
|27Q262|             74|   29674|   Queens|
|30Q501|            174|   85956|   Queens|
|21K337|             54|   16956| Brooklyn|
|30Q502|            148|   58904|   Queens|
|10X445|            731|  474419|    Bronx|
|04M435|            375|  170625|Manhattan|
|10X284|             54|   20790|    Bronx|
|16K498|             46|   16468| Brooklyn|
|13K483|            105|   40845| Brooklyn|
|11X249|             74|   28268|    Bronx|
|18K642|             40|   14960| Brooklyn|
|03M479|            261|  154512

In [7]:
fianl_data = join_data.groupby('Borough').sum()
fianl_data.show()

+-------------+--------------------+-------------+
|      Borough|sum(Num Test Takers)|sum(Sum Math)|
+-------------+--------------------+-------------+
|       Queens|               11274|      4864971|
|     Brooklyn|               12003|      5097081|
|Staten Island|                3005|      1378080|
|        Bronx|                6396|      2639824|
|    Manhattan|                8121|      3827992|
+-------------+--------------------+-------------+



In [8]:
result = fianl_data.map(lambda x: (x[0], x[2]/x[1]))
result.collect()

[(u'Queens', 431),
 (u'Brooklyn', 424),
 (u'Staten Island', 458),
 (u'Bronx', 412),
 (u'Manhattan', 471)]